Importing required libraries

In [1]:
import numpy as np
from random import shuffle
import pandas as pd
X_filename = "X.csv"
Y_filename = "y.csv"

In [14]:
#Utility functions:

class dLocalMAP:
    """
    See Anderson (1990, 1991)
    'Categories' renamed 'clusters' to avoid confusion.
    Discrete version.
    
    Stimulus format is a list of integers from 0 to n-1 where n is the number
    of possible features (e.g. [1,0,1])
    
    args: c, alphas
    """
    
    def __init__(self, args):
        self.partition = [[]]
        self.c, self.alpha = args
        self.alpha0 = [sum(f) for f in self.alpha]
        self.N = 0
    
    def probClustVal(self, k, i, val):
        """Find P(j|k)"""
        cj = len([x for x in self.partition[k] if x[i]==val])
        nk = len(self.partition)
        # return (cj + self.alpha[i][val])/(nk + self.alpha0[i])
        return (cj + self.alpha[i][val])/(nk + self.alpha0[i])
    
    def condclusterprob(self, stim, k):
        """Find P(F|k)"""
        pjks = []
        for i in range(len(stim)):
            cj = len([x for x in self.partition[k] if x[i]==stim[i]])
            nk = len(self.partition[k])
            # pjks.append( (cj + self.alpha[i][stim[i]])/(nk + self.alpha0[i]) )
            # print(i, self.alpha0)
            pjks.append( (cj + self.alpha[i][stim[i]])/(nk + self.alpha0[i]) )
        return np.product( pjks )
        
    
    def posterior(self, stim):
        """Find P(k|F) for each cluster"""
        pk = np.zeros( len(self.partition) )
        pFk = np.zeros( len(self.partition) )
        
        # existing clusters:
        for k in range(len(self.partition)):
            pk[k] = self.c * len(self.partition[k])/ ((1-self.c) + self.c * self.N)
            if len(self.partition[k])==0: # case of new cluster
                pk[k] = (1-self.c) / (( 1-self.c ) + self.c * self.N)
            pFk[k] = self.condclusterprob( stim, k)
        
        # put it together
        pkF = (pk*pFk) # / sum( pk*pFk )
        
        return pkF
    
    def stimulate(self, stim):
        """Argmax of P(k|F) + P(0|F)"""
        winner = np.argmax( self.posterior(stim) )
        # print( "Stim: ", stim)
        # print("Partition: ", self.partition)
        # print( self.posterior(stim))
        
        if len(self.partition[winner]) == 0:
            self.partition.append( [] )
        self.partition[winner].append(stim)
        
        self.N += 1
    
    def query(self, stimulus):
        """Queried value should be -1."""
        qdim = -1
        for i in range(len(stimulus)):
            if stimulus[i] < 0:
                if qdim != -1:
                    raise (Exception, "ERROR: Multiple dimensions queried.")
                qdim = i
        
        self.N = sum([len(x) for x in self.partition])
        
        pkF = self.posterior(stimulus)
        pkF = pkF[:-1] / sum(pkF[:-1]) # eliminate `new cluster' prob
        
        pjF = np.array( [sum( [ pkF[k] * self.probClustVal(k, qdim, j) for k in range(len(self.partition)-1)] )  for j in range(len( self.alpha[qdim] ))] )
        
        return pjF / sum(pjF)


def testlocalmapD(alpha):
    """
    Tests the Anderson's ratinal model using the Medin & Schaffer (1978) data.
    
    This script will print out the probability that each item belongs to each
    of the existing clusters or to a new cluster, and the model assign it to
    the most likely cluster. To see that the model is working correctly, you
    can follow along with Anderson (1991), which steps through in the same way.
    """
    # stims = [[1, 1, 1, 1, 1], # Medin & Schaffer (1978)
    #          [1, 0, 1, 0, 1], 
    #          [1, 0, 1, 1, 0],
    #          [0, 0, 0, 0, 0],
    #          [0, 1, 0, 1, 1],  
    #          [0, 1, 0, 0, 0]]
    # These are the classic Shepard Type II and Type IV datasets.
    # Uncomment the one you want to try out; you might want to uncomment
    # shuffling the stims too if you don't care about order.
    #stims = [[0, 0, 0, 0], [0, 0, 1, 0], [1, 1, 0, 1], [1, 1, 1, 1], [1, 0, 0, 0], [1, 0, 1, 1], [0, 1, 0, 0], [0, 1, 1, 1]] # Type IV
    #stims = [[0, 0, 0, 0], [0, 0, 1, 0], [1, 1, 0, 0], [1, 1, 1, 0], [1, 0, 0, 1], [1, 0, 1, 1], [0, 1, 0, 1], [0, 1, 1, 1]] # Type II

    stims = pd.read_csv(X_filename, names=['weight', 'height', 'label'])
    stims = stims.to_numpy()
    y_stim = pd.read_csv(Y_filename, names=['weight', 'height'])
    y_stim = y_stim.to_numpy()
    for stim in stims:
        #sprint(stim)
        stim[2] = int(stim[2]) -  1
        #print(stim[2])

    for _ in range(1):
        # alpha = [[0.05]*95, [0.05]*95, [0.5, 0.5, 0.5]]
        model = dLocalMAP([.5, alpha])
        stims = list(stims)
        shuffle(stims)
        
        for s in stims:
            model.stimulate(list(s))
        #print("Partitions:",model.partition)
        
        #print ("Prob vals for 0,0,0,0,?", model.query( [0]*(len(stims[0])-1) + [-1] ) )
        output_dict = {}
        for st in y_stim:
            #print(st)
            op = model.query( list(st) + [-1] )
            output_dict[str(st)] = op
            #print (f"Prob vals for {st} is", )

        #print ("Prob vals for 0,0,0,0,?", model.query( [85,69] + [-1] ) )
    return output_dict





Assumptions: 
1) he is polite, and is far more likely to call someone big average than
large
2) He is more likely to use weight than height to make category
judgments about size.


We see that alpha_j is used in calculation of $P_i(j|k)$ which in turn is used to calculate $P_i(j|F)$, we see that $P_i(j|K) = \frac{c_j + \alpha_j}{n_k + \alpha_0}$.

Thus $\alpha_j$ controls how much value $P_i(j|K)$ will be, which means in a partition k, how likely is the value j. 

Giving high values of $\alpha_j$ for a input label like weigth or height will shift bias towards that label. Thus we set value of alpha's first list to be higher than 2nd list. 

Coming to third entry of alpha, this is the entry which decides bias among different label category(1, 2, 3). Here our assumption is that hes more likely to call someone big as average, thus we give more weightage to category 2 than category 3. We will try various combinations

Combination 1, vanilla parametes.

In [20]:
output = testlocalmapD(alpha=[[0.05]*93, [0.05]*85, [1, 1, 1]])
for key in output.keys():
    print(f"Prediction for {key}: {np.argmax(output[key])+1}")


Prediction for [74 67]: 2
Prediction for [69 63]: 2
Prediction for [92 81]: 2
Prediction for [64 61]: 2
Prediction for [66 84]: 2
Prediction for [76 68]: 2
Prediction for [61 58]: 1
Prediction for [64 76]: 2
Prediction for [68 66]: 2
Prediction for [34 61]: 2


Most of the labels predicted are two because of higher number of 2 labels in the dataset

Combination 2, we give weight label more bias, and give category 2 higher preference than category 3.

In [18]:
output = testlocalmapD(alpha=[[0.05]*93, [0.05]*85, [1, 1.5, 0.5]])
for key in output.keys():
    print(f"Prediction for {key}: {np.argmax(output[key])+1}")


Prediction for [74 67]: 2
Prediction for [69 63]: 2
Prediction for [92 81]: 2
Prediction for [64 61]: 2
Prediction for [66 84]: 2
Prediction for [76 68]: 2
Prediction for [61 58]: 1
Prediction for [64 76]: 2
Prediction for [68 66]: 2
Prediction for [34 61]: 2


Combination 3: Label 1 and 2 have same bias, label 3 has low bias

In [19]:
output = testlocalmapD(alpha=[[0.05]*93, [0.05]*85, [1.5, 1.5, 0.5]])
for key in output.keys():
    print(f"Prediction for {key}: {np.argmax(output[key])+1}")


Prediction for [74 67]: 2
Prediction for [69 63]: 2
Prediction for [92 81]: 2
Prediction for [64 61]: 2
Prediction for [66 84]: 2
Prediction for [76 68]: 2
Prediction for [61 58]: 1
Prediction for [64 76]: 2
Prediction for [68 66]: 2
Prediction for [34 61]: 2
